In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
import seaborn as sns
import os, sys, warnings
from time import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt, ticker
from torch import optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import tqdm

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [ ]:
class MLP(nn.Module):

    def __init__(self,  fc1_num, fc2_num, fc3_num, dropout_rate):
        super(MLP, self).__init__()
        self.fc1_num = fc1_num
        self.fc2_num = fc2_num
        self.fc3_num = fc3_num

        # 两个全连接层
        self.fc1 = nn.Linear(fc1_num, fc2_num) # 702 -> 30
        self.fc2 = nn.Linear(fc2_num, 1)
        # self.fc3 = nn.Linear(fc3_num, 1)# 30 -> 1
        # 激活函数
        self.relu = nn.ReLU()
        # dropout
        self.dropout = nn.Dropout(dropout_rate)
        self.flatten = nn.Flatten()
        # 初始化权重
        self._init_weights()

    def _init_weights(self):
        # 使用 xavier 的均匀分布对 weights 进行初始化
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        # 使用正态分布对 bias 进行初始化
        nn.init.normal_(self.fc1.bias, std=1e-6)
        nn.init.normal_(self.fc2.bias, std=1e-6)

    def forward(self, data):
        data = self.flatten(data)
        data = self.fc1(data) # N*30
        data = self.relu(data)
        data = self.dropout(data)
        data = self.fc2(data)
        # data = self.relu(data)
        # data = self.dropout(data)
        # data = self.fc3(data)
        # 线性激活函数，无需再进行激活
        data = data.to(torch.float)

        return data


class FactorData():

    def __init__(self, train_x, train_y):
        self.len = len(train_x)
        self.x_data = train_x
        self.y_data = train_y

    def __getitem__(self, index):
        """
        指定读取数据的方式：根据索引 index 返回 dataset[index]

        """
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len


In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
def train_mlp(model, train_x, train_y, test_x, test_y, lr=1e-4, epoch=30, batch_size=1000, mom=0.9):
    # 1. 加载数据
    train_dataset = FactorData(train_x, train_y)
    test_dataset = FactorData(test_x, test_y)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    # 2. 模型训练
    weight_list, bias_list = [], []
    for name, p in model.named_parameters():
        if 'bias' in name: 
            bias_list += [p] # 将所有的 bias 参数放入 bias_list 中
        else:
            weight_list += [p] # 将所有的 weight 参数放入 weight_list 中
    
    # 定义损失函数和优化器
    criterion = nn.MSELoss()
    test_criterion = nn.L1Loss()

    optimizer = optim.RMSprop(
        [{'params': weight_list, 'weight_decay': 1e-5},
         {'params': bias_list, 'weight_decay': 0}],
        lr=lr,
        momentum=mom)
    
    train_loss_list = []
    test_loss_list = []
    best_test_epoch, best_test_loss = 0, np.inf
    seed = 0

    # 开始训练
    for epoch in range(1, epoch+1):
        # 测试模式
        test_loss = 0
        model.eval()
        test_batch_num = 0
        with torch.no_grad():
            for data, label in tqdm(test_loader, f'Epoch {epoch}-test ', leave=False):
                test_batch_num += 1
                data, label = data.to(torch.float), label.to(torch.float)
                # 得到测试集的预测值
                y_pred = model(data)
                # 计算损失
                loss = test_criterion(y_pred, label)
                # 将损失值加入到本轮测试的损失中
                test_loss += loss.item()
    
    # 在训练集中训练模型
    # .train() 参考 https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch
    # .zero_grad() 参考 https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch

    train_loss = 0
    model.train()  
    train_batch_num = 0
    for data, label in tqdm(train_loader, f'Epoch {epoch}-train', leave=False):
        train_batch_num += 1
        # 准备数据
        data, label = data.to(torch.float), label.to(torch.float)
        # 得到训练集的预测值
        out_put = model(data)
        # 计算损失
        loss = criterion(out_put, label)
        # 将损失值加入到本轮训练的损失中
        train_loss += loss.item()
        # 梯度清零
        optimizer.zero_grad() 
        # 反向传播求解梯度
        loss.backward()
        # 更新权重参数
        optimizer.step()

    train_loss_list.append(train_loss/train_batch_num)
    test_loss_list.append(test_loss/test_batch_num)

    return model 

In [ ]:
# 将数据载入到 DataLoader 中
batch_size = 1000
train_data = FactorData(train_x.values, train_y.values)
# train_data = FactorData(train_x_converted, train_y_converted)
train_loader = DataLoader(dataset=train_data,
                          batch_size=batch_size,
                          shuffle=False)  # 不打乱数据集
test_data = FactorData(test_x.values, test_y.values)
# test_data = FactorData(test_x_converted, test_y_converted)
test_loader = DataLoader(dataset=test_data,
                         batch_size=batch_size,
                         shuffle=False)  # 不打乱数据集

# 构建模型
# alphanet = AlphaNet(combination=combination, combination_rev=combination_rev,
#                     index_list=index_list, fc1_num=8820, fc2_num=55, fc3_num=0, dropout_rate=0.2)
mlp = MLP(fc1_num=120, fc2_num=60, fc3_num=0, dropout_rate=0.2)
weight_list, bias_list = [], []
for name, p in mlp.named_parameters():
    # 将所有的 bias 参数放入 bias_list 中
    if 'bias' in name:
        bias_list += [p]
    # 将所有的 weight 参数放入 weight_list 中
    else:
        weight_list += [p]

# weight decay: 对所有 weight 参数进行 L2 正则化
optimizer = optim.RMSprop([{'params': weight_list, 'weight_decay': 1e-5},
                           {'params': bias_list, 'weight_decay': 0}],
                          lr=1e-4,
                          momentum=0.9)

# 损失函数为均方误差 MSE
criterion = nn.MSELoss()
test_criterion = nn.L1Loss()
epoch_num = 50
train_loss_list = []
test_loss_list = []
best_test_epoch, best_test_loss = 0, np.inf
seed = 0
for epoch in range(1, epoch_num+1):
    # 测试模式
    test_loss = 0
    mlp.eval()
    test_batch_num = 0
    with torch.no_grad():
        for data, label in tqdm(test_loader, f'Epoch {epoch}-test ', leave=False):
            test_batch_num += 1
            data, label = data.to(torch.float), label.to(torch.float)
            # 得到测试集的预测值
            y_pred = mlp(data)
            # 计算损失
            loss = test_criterion(y_pred, label)
            # 将损失值加入到本轮测试的损失中
            test_loss += loss.item()

    train_loss = 0
    # 在训练集中训练模型
    mlp.train()  # 关于。train() 的作用，可以参考 https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch
    train_batch_num = 0
    for data, label in tqdm(train_loader, f'Epoch {epoch}-train', leave=False):
        train_batch_num += 1
        # 准备数据
        data, label = data.to(torch.float), label.to(torch.float)
        # 得到训练集的预测值
        out_put = mlp(data)
        # 计算损失
        loss = criterion(out_put, label)
        # 将损失值加入到本轮训练的损失中
        train_loss += loss.item()
        # 梯度清零
        optimizer.zero_grad() # 关于。zero_grad() 的作用，可以参考 https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch
        # 反向传播求解梯度
        loss.backward()
        # 更新权重参数
        optimizer.step()




    train_loss_list.append(train_loss/train_batch_num)
    test_loss_list.append(test_loss/test_batch_num)
print(train_loss_list)
print(test_loss_list)

